# Install Libraries

Skip to the git clone oart if using locally

In [0]:
!pip install fastai

    100% |████████████████████████████████| 153kB 10.4MB/s 
    100% |████████████████████████████████| 112kB 32.6MB/s 
    100% |████████████████████████████████| 61kB 26.3MB/s 
    100% |████████████████████████████████| 591.8MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6105e000 @  0x7fd99ce352a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
  Running setup.py bdist_wheel for bottleneck ... -

# Load Data


In [0]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

sample_rate, samples = wavfile.read('coppersmith_barbet/Coppersmith-Barbet-(Psilopogon-haemacephalus)--xenocanto(54).wav')
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

plt.pcolormesh(times, frequencies, spectrogram)
plt.imshow(spectrogram)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Change working directory 

import os
os.chdir("drive/My Drive/fastai/cheetah_leopard")

# Training

In [0]:
path = Path('data')
path.ls()

In [0]:
classes = ['cheetah', 'leopard']

In [0]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_workers=8)

In [0]:
np.random.seed(4)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=229, num_workers=4).normalize(imagenet_stats)

In [0]:
data.show_batch(rows=3, figsize=(7,8))

In [0]:
print(data.classes)
len(data.classes),data.c

## Training: resnet50

Start training our model. We will use a convolutional neural network backbone and a fully connected head with a single hidden layer as a classifier. 

We will train for 5 epochs (5 cycles through all our data).

In [0]:
learn = create_cnn(data, models.resnet50, metrics=error_rate)

In [0]:
learn.fit_one_cycle(5)

In [0]:
learn.save('model')

## Interpretation

In [0]:
learn.load('model')

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

## Using Inferences

In [0]:
defaults.device = torch.device('cpu')

In [0]:
img = open_image(path/'leopard'/'6. sn-leopard.jpg')
img

In [0]:
classes = ['leopard', 'cheetah']
data2 = ImageDataBunch.single_from_classes(path, classes, tfms=get_transforms(), size=229).normalize(imagenet_stats)
learn = create_cnn(data2, models.resnet50)
learn.load('model')

In [0]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class